![Spark Logo](http://spark-mooc.github.io/web-assets/images/ta_Spark-logo-small.png)  ![Python Logo](http://spark-mooc.github.io/web-assets/images/python-logo-master-v3-TM-flattened_small.png)
# Contando palabras: Construye una aplicacion que cuente palabras de forma eficiente

Este laboratorio usara las tecnologias descritas en los materiales del curso sobre Spark para desarrollar una aplicacion de conteo de palabras. 

Con el uso masivo de Internet y las redes sociales, el volumen de texto no estructurado esta creciendo dramaticamente, y Spark es una gran herramienta para analizar este tipo de datos. En esta PEC, vamos a escribir codigo para encontrar las palabras mas comunes en un texto generado en latin, el ya conocido [Lorem Ipsum](https://www.lipsum.com/).


Lo mas interesante de la forma de trabajar en esta practica es que podria escalarse para, por ejemplo, encontrar las palabras mas comunes en Wikipedia.

## Durante esta PEC vamos a cubrir:

* *Parte 1:* Creación de un RDD y un pair RDD
* *Parte 2:* Contar palabras usando un pair RDD
* *Parte 3:* Encontrar las palabras individuales y su frecuencia de aparicion media
* *Parte 4:* Aplicar las funcionalidades desarrolladas a un archivo de texto* 
* *Parte 5:* Calcular algunos estadisticos*


> Como referencia a todos los detalles de los metodos que se usan en esta practica usar:
> * [API Python de Spark](https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD)

## Parte 1: Creacion de un RDD y un pair RDDs

En esta seccion, exploraremos como crear RRDs usando `parallelize` y como aplicar pair RDDs al problema del conteo de palabras.

### (0) Configuración del entorno python + spark

In [1]:
import findspark
findspark.init()
import pyspark
import random
sc = pyspark.SparkContext(master="local", appName="Gonzalezjulvez")

### (1a) Creación de un RDD
Empezemos generando un RDD a partir de una lista de Python y el metodo `sc.parallelize`. Luego mostraremos por pantalla el tipo de la variable generada.

In [2]:
wordsList = ['cat', 'elephant', 'rat', 'rat', 'cat']
wordsRDD = sc.parallelize(wordsList, 4)
# Print out the type of wordsRDD
type(wordsRDD)

pyspark.rdd.RDD

### (1b) Crear el plural de las palabras y testear

Vamos a utilizar una transformacion `map()` para incorporar la letra 's' a cada uno de los strings almacenados en el RDD que acabamos de crear. Vamos a definir una funcion de Python que devuelva una palabra, que se le ha pasado como parametro, incorporando una "s" al final de la misma. 

In [5]:
def makePlural(word):
    return word + 's'

makePlural('cat')

'cats'

### (1c) Aplicar `makePlural` a nuestro RDD

Ahora es el momento de aplicar nuestra funcion `makePlural()` a todos los elementos del RDD usando una transformacion [map()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.map). Posteriormente ejecutar la accion [collect()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.collect) para obtener el RDD transformado.

In [6]:
pluralRDD = wordsRDD.map(makePlural)
pluralRDD.collect()

['cats', 'elephants', 'rats', 'rats', 'cats']

### (1d) Ejecutar una funcion `lambda` en un `map`

Vamos a crear el mismo RDD usando una `lambda` function en lugar de una funcion con nombre.

In [7]:
pluralLambdaRDD = wordsRDD.map(lambda word : makePlural(word))
print(pluralLambdaRDD.collect())

['cats', 'elephants', 'rats', 'rats', 'cats']


### (1e) Numero de caracteres de cada una de las palabras

Ahora vamos a usar un `map()` y una funcion lambda `lambda` para obtener el numero de caracteres de cada palabra. Usaremos `collect` para guardar este resultado directamente en una variable.

In [8]:
pluralLengths = (pluralRDD.map(lambda word : len(word)).collect())
print(pluralLengths)

[4, 9, 4, 4, 4]


### (1f) Pair RDDs

El siguiente paso para completar nuestro programa de conteo de palabras en crear un nuevo tipo de RDD, llamado pair RDD. Un pair RDD es un RDD donde cada elemento es un tupla del estilo `(k, v)` donde `k` es la clave y `v` es su valor correspondiente. En este ejemplo, crearemos una pair RDD consistente en tuplas con el formato `('<word>', 1)` para cada elemento de nuestro RDD basico.

Podemos crear nuestro pair RDD usando una transformacion `map()` con una `lambda()` function que cree un nuevo RDD.

In [9]:
wordPairs = wordsRDD.map(lambda word : (word,1))
print(wordPairs.collect())

[('cat', 1), ('elephant', 1), ('rat', 1), ('rat', 1), ('cat', 1)]


## Parte 2: Contar palabras usando un pair RDD

Ahora, contaremos el numero de veces que una palabra en particular aparece en el RDD. Esta operacion se puede realizar de una infinidad de maneras, pero algunas seran mucho menos eficientes que otras.

Un solucion muy sencilla seria usar `collect()` sobre todos los elementos devolverlos al driver y alli contarlos. Mientras esta forma de trabajar podria funcionar con textos relativamente cortos, nosotros lo que queremos es poder trabajar con textos de cualquier longitud. Adicionalmente, ejecutar todo el calculo en el driver es mucho mas lento que ejecutarlo en paralelo en los workers. Por estos motivos, en esta practica usaremos operaciones paralelizables.

%md
### (2a) Usando `groupByKey()`
Una primera solucion a nuestro problema, luego veremos que hay otras mucho mas eficientes, se podria basar en la transformacion [groupByKey()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.groupByKey). Como su nombre indica, la transformacion `groupByKey()` agrupa todos los elementos de un RDD que compartan la misma clave en una unica lista dentro de una de las particiones.

Esta operacion plantea dos problemas:
  + Esta operacion necesita mover todos los valores dentro de la particion adecuada. Esto satura la red. 
  + Las listas generadas pueden llegar a ser muy grandes llegando incluso a saturar la memoria de alguno de los trabajadadores
  
Utiliza `groupByKey()` para generar un pair RDD del tipo `('word', iterator)`.

In [10]:
wordsGrouped = wordPairs.groupByKey()
for key, value in wordsGrouped.collect():
    print('{0}: {1}'.format(key, list(value)))

cat: [1, 1]
elephant: [1]
rat: [1, 1]


### (2b) Utiliza `groupByKey()` para obtener los conteos

Usando la transformacion `groupByKey()` crea un RDD que contenga 2 elementos, donde cada uno de ellos sea un par palabra (clave) iterador de Python (valor).

Luego suma todos los valores de iterador usando una transformacion `map()`. El resultado debe ser un pair RDD que contenga las parejas (word, count).

In [11]:
wordCountsGrouped = wordsGrouped.map(lambda x : (x[0], len(x[1])))
print(wordCountsGrouped.collect())

[('cat', 2), ('elephant', 1), ('rat', 2)]


### (2c) Conteo usando `reduceByKey`

Una mejor solucion es comenzar desde un pair RDD y luego usar la transformacion [reduceByKey()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.reduceByKey) para crear un nuevo pair RDD. La transformacion `reduceByKey()` agrupa todas las parejas que comparten la misma clave. Posteriormente aplica la funcion que se le pasa por parametro agrupando los valores de dos en dos. Este proceso se repite iterativamente hasta que obtenemos un unico valor agregado para cada una de las claves del pair RDD. `reduceByKey()` opera aplicando la funcion primero dentro de cada una de las particiones de forma independiente, y posteriormente unicamente comparte los valores agregados entre particiones diferentes, permitiendole escalar de forma eficiente ya que no tiene necesidad de desplazar por la red una gran cantidad de datos.

In [12]:
wordCounts = wordPairs.reduceByKey(lambda x,y : x+y)
print (wordCounts.collect())

[('cat', 2), ('elephant', 1), ('rat', 2)]


### (2d) Ahora todo junto

La version mas compleja del codigo ejecuta primero un `map()` sobre el pair RDD, la transformacion `reduceByKey()`, y finalmente la accion `collect()` en una unica linea de codigo.

In [13]:
wordCountsCollected = (wordsRDD
                       .map(lambda word : (word,1))
                       .reduceByKey(lambda x,y: x+y)
                       .collect())
print(wordCountsCollected)

[('cat', 2), ('elephant', 1), ('rat', 2)]


## Parte 3: Encontrar las palabras individuales y su frecuencia de aparicion media

### (3a) Palabras unicas

Calcular el numero de palabras unicas en `wordsRDD`. Puedes utitlziar otros RDDs que hayas creado en esta practica si te resulta mas sencillo.

In [15]:
uniqueWords = wordCounts.count()
print(uniqueWords)

3


### (3b) Calular la media usando `reduce()`

Encuentra la frequencia media de aparicion de palabras en `wordCounts`.

Utiliza la accion `reduce()` para sumar los conteos en `wordCounts` y entonces divide por el numero de palabras unicas. Para realizar esto primero aplica un `map()` al pair RDD `wordCounts`, que esta formado por tuplas con el formato (key, value), para convertirlo en un RDD de valores.

In [16]:
from operator import add
totalCount = (wordCounts
              .map(lambda x: x[1])
              .reduce(add))
average = totalCount / float(wordCounts.count())
print(totalCount)
print(round(average, 2))

5
1.67


## Parte 4: Aplicar las funcionalidades desarrolladas a un archivo de texto

Para esto hemos de construir una funcion `wordCount`, capaz de trabajar con datos del mundo real que suelen presentan problemas como el uso de mayusculas o minusculas, puntuacion, acentos, etc. Posteriormente, cargar los datos de nuestra fuente de datos y finalmente, calular el conteo de palabras sobre los datos procesados.

### (4a) funcion `wordCount`

Primero, define una funcion para el conteo de palabras. Deberias reusar las tecnicas que has visto en los apartados anteriores de esta practica. Dicha funcion, ha de tomar un RDD que contenga una lista de palabras, y devolver un pair RDD que contenga todas las palabras con sus correspondientes conteos.

In [17]:

def wordCount(wordListRDD):
   
    return wordListRDD.map(lambda x : (x,1)).reduceByKey(lambda x,y:x+y)

print(wordCount(wordsRDD).collect())

[('cat', 2), ('elephant', 1), ('rat', 2)]


### (4b) Mayusculas y puntuacion

Los ficheros del mundo real son mucho mas complejos que los que hemos estado usando en esta PAC. Algunos de los problemas que son necesarios de solucionar son:
  + Las palabras deben de contarse independientemente de si estan en mayuscula o minuscula (por ejemplo, Spark y spark deberian contarse como la misma palabra).
  + Todos los signos de puntuacion han de eliminarse.
  + Cualquier espacio al principio o al final de la palabra ha de eliminarse.
  

In [18]:
import re
def removePunctuation(text):
    lowertext = text.lower()
    lowertext_rm = re.sub(r'[^a-z0-9\s]','',lowertext)
    lowertext_final = lowertext_rm.strip()
    
    return lowertext_final
    
print(removePunctuation('Hi, you!'))
print(removePunctuation(' No under_score!'))
print(removePunctuation(' *      Remove punctuation then spaces  * '))

hi you
no underscore
remove punctuation then spaces


### (4c) Cargar un fichero de texto

Para la siguiente parte, usaremos el texto ya mencionado Lorem Ipsum generado para la pràctica.Para convertir un fichero de texto en un RDD, usaremos el metodo `SparkContext.textFile()`. Tambien usaremos la funcion que acabamos de crear `removePunctuation()` dentro de una transformacion `map()` para eliminar todos los caracteres no alphabeticos, numericos or espacios. Dado que el fichero es bastante grandre, usaremos `take(15)`, de forma que tan solo imprimiremos por pantalla las 15 primeras lineas.

In [20]:
pwd

'/home/gonzalezjulvez/Documentos/Projects_Github/Projects/Pyspark'

In [30]:
# Tan solo ejecuta este codigo
import os.path

fileName = os.path.join('/home/gonzalezjulvez/Documentos/Projects_Github/Projects/Pyspark/data/', 'Shakespeare.txt')

ShakeRDD = sc.textFile(fileName, 8).map(removePunctuation).filter(lambda x: len(x)>0)
ShakeRDD.take(10)

['project gutenbergs the complete works of william shakespeare by william shakespeare',
 'this ebook is for the use of anyone anywhere in the united states and',
 'most other parts of the world at no cost and with almost no restrictions',
 'whatsoever  you may copy it give it away or reuse it under the terms',
 'of the project gutenberg license included with this ebook or online at',
 'wwwgutenbergorg  if you are not located in the united states youll',
 'have to check the laws of the country where you are located before using',
 'this ebook',
 'title the complete works of william shakespeare',
 'author william shakespeare']

### (4d) Extraer las palabras de las lineas

Antes de poder usar la funcion `wordcount()`, hemos de solucionar dos problemas con el formato del RDD:
  + El primer problema es que necesitamos dividir cada linea por sus espacios. ** Esto lo solucionaremos en el apartado (4d). **
  + El segundo problema es que necesitamos filtar las lineas completamente vacias. ** Esto lo solucionaremos en el apartado (4e). **


In [31]:
ShakeWordsRDD = ShakeRDD.flatMap(lambda x: x.split(" "))
ShakeWordsCount = ShakeWordsRDD.count()
print(ShakeWordsRDD.top(5))
print(ShakeWordsCount)

['zwounds', 'zwounds', 'zwounds', 'zwounds', 'zwounds']
976877


### (4e) Calcula palabras distintas

El siguiente paso es contar cuantas palabras distintas contiene nuestro texto. Puedes usar las transformaciones map() y reduceByKey() ya utilizadas anteriormente.

In [32]:
distintWordsMapRDD = ShakeWordsRDD.map(lambda x:(x,1)).reduceByKey(lambda x,y:x+y)

distintWordsRDD=distintWordsMapRDD.keys().distinct()

print(distintWordsRDD.take(8))   
print(distintWordsRDD.count())


['of', 'other', 'whatsoever', '', 'gutenberg', 'are', 'check', 'where']
31566


### (4f) Cuenta las palabras

Ahora que tenemos un RDD que contiene solo palabras. El siguiente paso es aplicar la funcion `wordCount()` para producir una lista con los conteos de palabras. Podemos ver las 15 mas comunes usando la accion `takeOrdered()`; sin embargo, como los elementos del RRD son pares, necesitamos una funcion especial que ordene los pares de la forma correcta.

Usa las funciones  `wordCount()` y `takeOrdered()` para obtener las 15 palabras mas comunes junto con sus conteos.

In [33]:
top15WordsAndCounts = wordCount(ShakeWordsRDD).takeOrdered(15, key = lambda x: -x[1])
print(top15WordsAndCounts)

[('the', 30205), ('and', 28386), ('i', 21949), ('to', 20923), ('of', 18822), ('a', 16182), ('', 15571), ('you', 14437), ('my', 13180), ('in', 12232), ('that', 11776), ('is', 9713), ('not', 9066), ('with', 8528), ('me', 8263)]


## Parte 5: Calcular algunos estadisticos

Usando las mismas tecnicas que has aplicado en los ejercicios anteriores responde a las siguientes preguntas:

### (5a) ¿Cual es la longitud media de todas las palabras (no repetidas) que aparecen en el texto?

In [34]:
len_allwords = distintWordsRDD.map(lambda x : len(x)).reduce(add)

average_allwords = len_allwords/float(distintWordsRDD.count())

print("La longitud media de las palabras es " + str(round(average_allwords,2)))

La longitud media de las palabras es 7.41


### (5b) ¿Cuantas palabras distintas hay con al menos una 'a' y al menos una 'u' ?

In [35]:
Words_a_u = distintWordsRDD.filter(lambda x: re.match(r'(\b\w*[a]\w*[u]\w*\b)|(\b\w*[u]\w*[a]\w*\b)',x))

print("En el texto seleccionado nos encontramos con {} palabras diferentes que al menos contienen una a y u.".format(Words_a_u.count()))

En el texto seleccionado nos encontramos con 2600 palabras diferentes que al menos contienen una a y u.


### (5c) ¿Cuál es la palabra que más se repite y la que menos?

In [36]:
MapReduceWords = ShakeWordsRDD.map(lambda x: (x,1)).reduceByKey(lambda x,y:x+y)
WordsAndCounts= MapReduceWords.takeOrdered(MapReduceWords.count(),key = lambda x: -x[1])


In [37]:
print("La palabra mas repetida sería '{}' con {} veces".format(WordsAndCounts[0][0],WordsAndCounts[0][1]))
print("La palabra menos repetida sería '{}' con {} veces".format(WordsAndCounts[-1][0],WordsAndCounts[-1][1]))

La palabra mas repetida sería 'the' con 30205 veces
La palabra menos repetida sería 'originator' con 1 veces
